In [19]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

import numpy as np

## Ссылки с открывающей страницы

In [14]:
url = 'https://outmaxshop.ru/snickers'

data = requests.get(url).text

soup_op = BeautifulSoup(data, 'html.parser')

links_op = soup_op.find('div', {'class': 'catalog-product__list'}).find_all('a')

links_op = [i['href'] for i in links_op]



## Функция, для получения ссылок на карточки товаров

In [8]:
def find_links(number):
    url = 'https://outmaxshop.ru/snickers?start=' + str(number)
    
    data = requests.get(url).text

    soup = BeautifulSoup(data, 'html.parser')
    
    a = soup.find('div', {'class': 'catalog-product__list'}).find_all('a')
    
    links = [i['href'] for i in a]
    
    return links

## Функция, для получения описания товара

In [9]:
def find_description(soup):
    b = soup.find('table', {'class': 'product-chars'})
    c = soup.find('table', {'class': 'product-chars'})
    if b is None or c is None:
        return [np.nan] * 5
    
    b = b.find_all('td', {'class': 'product-chars__text'})
    c = c.find_all('td', {'class': 'product-chars__name'}) 
    names = [(i.text).strip() for i in c]  
    desc = [(i.text).strip() for i in b]
    dic = {i: j for i,j in zip(names, desc)}
    info = []
    for i in ['Материал верха', 'Модель', 'Категория', 'Сезон', 'Цвет']:
        if i in dic:
            info.append(dic[i])
        else:
            info.append(np.nan)
    
    return info

## Функция для получения брэнда товара

In [10]:
def find_brand(soup):
    brand = soup.find('a', {'class': 'product__brand'})
    if brand is None:
        return [np.nan]
    else:
        brand = brand.find_all('meta')
        return [brand[0]['content']]

## Функция, для получения цен товара

In [11]:
def find_price(soup):
    prices_old = (soup.find_all('div', {'class': 'product__price product__price--line'}))[0].text
    prices_current = (soup.find_all('div', {'class': 'product__price product__price--red'}))[0].text

    price_old = int((prices_old.strip()).replace('руб.', '').replace(' ', ''))
    price_current = int((prices_current.strip()).replace('руб.', '').replace(' ', ''))
    
    prices = [price_old, price_current]
    
    return prices

##  Функция, для получения размеров товара

In [12]:
def find_sizes(soup):
    all_sizes = soup.find('div', {'class': 'product-size__container'}).find_all('span', 'radio_attr_label')
    
    fin_all_sizes = ','.join([i.text for i in all_sizes])

    return [fin_all_sizes]

## Функция для получения html-кода страницы

In [13]:
def get_the_soup(link):
    url = 'https://outmaxshop.ru' + link
    
    data = requests.get(url).text
    
    soup = BeautifulSoup(data, 'html.parser')
    
    return soup

In [25]:
for i in links_op:
    print(i)
    try:
        soup = get_the_soup(i)
        
        brand = find_brand(soup)
        
        description = find_description(soup)
        
        
        prices = find_price(soup)
        
        sizes = find_sizes(soup)
        
        infor = brand + description + sizes + prices
        

        
        df.loc[len(df)] = infor
    except Exception:
        df.loc[len(df)] = [np.nan] * 9
        

/snickers/adidas-terrex-winter-12413?fixed=12413&fixed_cat=1789
/snickers/adidas-terrex-winter-12104?fixed=12104&fixed_cat=1789
/snickers/adidas-terrex-winter-12103?fixed=12103&fixed_cat=1789
/accessories/30129?fixed=30129&fixed_cat=1789
/snickers/climbing-34214
/snickers/delta-34213
/snickers/34210
/snickers/34208
/snickers/fashion-34169
/snickers/fashion-34168
/snickers/fashion-34167
/snickers/nike-classic-cortez-34139
/snickers/nike-air-jordan-4-retro-34138
/snickers/nike-air-jordan-1-low-34137
/snickers/nike-air-jordan-1-low-34136
/snickers/adidas-nite-jogger-34135
/snickers/salomon-speedcross-5-gtx-34133
/snickers/bates-34092
/snickers/delta-34091
/snickers/lowa-zephyr-gtx-34090
/snickers/nike-air-jordan-4-retro-33988
/snickers/kaws-x-nike-air-jordan-4-retro-33987
/snickers/new-balance-9060-33986
/snickers/new-balance-9060-33985
/snickers/new-balance-9060-33984
/snickers/new-balance-9060-33983
/snickers/new-balance-9060-33982
/snickers/bape-court-sta-low-33981
/snickers/adidas-ozw

In [24]:
for i in range(40,5090,40):
    needed_links = find_links(i)
    print(i)
    
    for j in needed_links:
        try:
            soup = get_the_soup(j)
        
            brand = find_brand(soup)
        
            description = find_description(soup)
        
        
            prices = find_price(soup)
        
            sizes = find_sizes(soup)
        
            infor = brand + description + sizes + prices
        
            df.loc[len(df)] = infor
        except Exception:
            df.loc[len(df)] = [np.nan] * 9

In [27]:
df.head(5500)

,brand,material,model,category,season,color,size,old_price,current_price
0,New Balance,"Кожа, текстиль",574,Спортивный стиль,Демисезон,Белый,"36,37,38,39,40",5790.0,4960.0
1,Nike,Кожа,M2K TEKNO,Спортивный стиль,Демисезон,Белый,"36,37,38,39,40",5690.0,4850.0
2,Adidas,"Текстиль, замша",Ozweego,Спортивный стиль,Демисезон,Розовый,"36,37,38,39",5690.0,4850.0
3,Adidas,Кожа,Forum Low,Спортивный стиль,Демисезон,Белый,"40,41,42,43,44,45",6290.0,5390.0
4,Nike,Кожа,D Venture Leather,Ботинки,Зима,Белый,"36,37,38,39,40",8290.0,7010.0
...,...,...,...,...,...,...,...,...,...
4585,Nike,Текстиль,Pegasus Hi,Ботинки,Демисезон,Черный,"40,41,42,43,44,45",6290.0,5390.0
4586,Premiata,"Замша, текстиль",Mick,Спортивный стиль,Демисезон,Разноцветный,"40,41,42,43,44,45",10890.0,9280.0
4587,Vans,"Текстиль, замша",Old Skool,Скейтборд,Лето,Синий,"40,41,42,43,44",5090.0,4310.0
4588,Nike,"Текстиль, кожа",Air Max 270 React ENG,Спортивный стиль,Демисезон,Бежевый,"41,42,43,44,45",6290.0,5390.0


Nice
